In [1]:
import urllib.request
from bs4 import BeautifulSoup
import os
import re

# Функция преобразовывает буквенный месяц
# (дата на сайте указана в формате ДД МЕСЯЦ (ГГГГ)) в числовой

def monthToDigit(month):
    if month == 'января':
        monthDigit = '01'

    elif month == 'февраля':
        monthDigit = '02'

    elif month == 'марта':
        monthDigit = '03'

    elif month == 'апреля':
        monthDigit = '04'

    elif month == 'мая':
        monthDigit = '05'

    elif month == 'июня':
        monthDigit = '06'

    elif month == 'июля':
        monthDigit = '07'

    elif month == 'августа':
        monthDigit = '08'

    elif month == 'сентября':
        monthDigit = '09'

    elif month == 'октября':
        monthDigit = '10'

    elif month == 'ноября':
        monthDigit = '11'

    elif month == 'декабря':
        monthDigit = '12'

    return monthDigit
# Функция поиска (мета)данных по тэгам


def meta(pageUrl):
    try:
        d = {}
        page = urllib.request.urlopen(pageUrl)
        text = page.read().decode('utf-8')
        soup = BeautifulSoup(text, 'html.parser')
        # поиск заголовка
        d['header'] = soup.find('div', {'class': 'f26'}).get_text()
        # поиск текста статьи
        tagText = soup.find('div', {'id': 'material_text'}).get_text()
        # чистим статью от тэгов html, которые попали в div
        woN = re.sub(r'\n', '', tagText)
        woR = re.sub(r'\r', '', woN)
        woT = re.sub(r'\t', '', woR)
        # чистый текст
        d['text'] = woT.strip()
        # здесь содержится информация об авторе и дате
        footer = soup.find('div', {'id': 'vnizu_a_n_v'})
        children = footer.findChildren('div')
        # строка с именем автора
        authorLine = str(children[0])
        # строка с датой
        dateLine = str(children[1])
        # поиск автора
        author = re.search(
            'Автор: (.*?)(((\.)|(\,)(.*?))|()|)</div>', authorLine)
        if author:
            d['author'] = author.group(1)
        # поиск даты
        date = re.search('Время: (.*?)</div>', dateLine)
        if date:
            # выделяем год
            year = re.search('[а-я]*? (....) г', date.group(1))
            if year:
                d['year'] = year.group(1)
            # но иногда его нет...
            else:
                d['year'] = 'None'
            # выделяем месяц
            month = re.search('.. ([а-я]*)[ ]*?(.... г)?', date.group(1))
            if month:
                digitMonth = month.group(1)
                d['month'] = monthToDigit(digitMonth.strip())
            # выделяем день
            day = re.search('(..) [а-я]*[ ]*?(.... г)?', date.group(1))
            if day:
                d['day'] = day.group(1)

            return d
    # на случай, если страница сломана
    except:
        print('Error at', pageUrl)

        
def crowler():
    # ссылка на странички
    commonUrl = 'http://www.vecherniy.com/wall?id='
    # огромный словарь всех статей с сайта
    # ключ - номер статьи на сайте,
    # значения - словарь с мета-данными каждой статьи
    bigD = {}
    # считаем статьи
    for i in range(1067, 1087):
        pageUrl = commonUrl + str(i)
        d = meta(pageUrl)
        if d:
            # если все прошло успешно, и ссылка не сломана -
            # добавляем в мета-данные адрес
            d['url'] = pageUrl
            bigD[i] = d
        # если нет - ничего не делаем
        else:
            pass
    return bigD

print(crowler())

{1067: {'header': 'Задержанный террорист ехал к нам', 'text': 'Сотрудники ФСБ встретили его на российской границе. По оперативной информации, мужчина возвращался в Нерюнгри, где прожил около десяти лет, чтобы «применить навыки, приобретенные в боевых лагерях запрещенной в России террористической организации Джабхат ан-Нусра».27-летний уроженец Киргизии Мирсухиджун Мирвохитов приехал в Нерюнгри в 2006 г. Здесь его отец держал бизнес по продаже продуктов, а большая семья помогала ему в этом. Бизнес был достаточно успешным: семья приобрела в городе три квартиры. Киргизская община в Нерюнгри вообще довольно обширна и в то же время достаточна закрыта, живет по своим законам. Трудно сказать, когда именно в голове молодого человека появились радикальные мысли. Силовики традиционно во всем винят Интернет.В 2016 году семья Мирвохитова оставила Россию. Сам он через Турцию перебрался в Сирию, где присоединился к боевикам. Киргиза приняли в лагерь боевой подготовки, научили стрелять и отправили на